In [1]:
!pip install pandas numpy

  Using cached numpy-2.3.2-cp313-cp313-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 562.5 kB/s eta 0:00:0000:0100:01
Using cached numpy-2.3.2-cp313-cp313-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (16.6 MB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [pandas]2m3/4 [pandas]

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [17]:
import pandas as pd
import numpy as np

np.random.seed(42)

In [19]:
n_users = 5000
min_txns, max_txns = 10, 20 
fraud_user_ratio = 0.01   
all_locations = ["Delhi", "Mumbai", "Hyderabad", "Chennai", "Bangalore", "Kolkata"]

In [4]:
user_profiles = pd.DataFrame({
    "user_id": np.arange(1000, 1000 + n_users),
    "avg_amount": np.random.uniform(300, 2000, n_users),
    "std_amount": np.random.uniform(50, 300, n_users),
    "home_location": np.random.choice(all_locations, n_users),
    "is_traveler": np.random.choice([0, 1], n_users, p=[0.8, 0.2])  # 20% frequent travelers
})

In [20]:
transactions = []
txn_id = 1

In [21]:
fraud_users = set(np.random.choice(user_profiles["user_id"], int(fraud_user_ratio * n_users), replace=False))

In [22]:
for _, user in user_profiles.iterrows():
    n_txns = np.random.randint(min_txns, max_txns + 1)
    last_ts = pd.Timestamp("2023-01-01").value // 10**9
    fraud_injected = 0  # count of frauds injected

    for i in range(n_txns):
        # Base transaction amount
        amount = np.random.normal(user["avg_amount"], user["std_amount"])
        amount = max(10, round(amount, 2))

        # Location (traveler logic)
        if user["is_traveler"] and np.random.rand() < 0.3:
            location = np.random.choice(all_locations)
        else:
            location = user["home_location"]

        # Device / Merchant
        device_id = np.random.randint(5000, 7000)
        merchant_id = np.random.randint(200, 1000)

        # Timestamp
        gap = np.random.randint(60, 60 * 60 * 24)  # 1 min – 1 day
        ts = last_ts + gap
        last_ts = ts
        txn_time = pd.to_datetime(ts, unit="s")

        # -------------------------
        # Fraud injection
        # -------------------------
        fraud = 0
        if user["user_id"] in fraud_users:
            # Inject 1–3 fraud txns per fraud user
            if fraud_injected < 2 and np.random.rand() < 0.3 and i > 2:
                fraud = 1
                fraud_injected += 1
                fraud_type = np.random.choice(["high_amount", "burst", "loc_device_change"])
                if fraud_type == "high_amount":
                    amount = round(user["avg_amount"] * np.random.uniform(8, 20), 2)
                elif fraud_type == "burst":
                    amount = round(user["avg_amount"] * np.random.uniform(5, 15), 2)
                    ts = last_ts + np.random.randint(1, 120)  # within 2 mins
                    last_ts = ts
                    txn_time = pd.to_datetime(ts, unit="s")
                elif fraud_type == "loc_device_change":
                    locs = [l for l in all_locations if l != user["home_location"]]
                    location = np.random.choice(locs)
                    device_id = np.random.randint(7000, 8000)
                    amount = round(user["avg_amount"] * np.random.uniform(5, 15), 2)

        # Optional random noise fraud for normal users (rare, 1%)
        elif np.random.rand() < 0.01:
            fraud = 1
            fraud_type = np.random.choice(["high_amount", "burst", "loc_device_change"])
            if fraud_type == "high_amount":
                amount = round(user["avg_amount"] * np.random.uniform(8, 20), 2)
            elif fraud_type == "burst":
                amount = round(user["avg_amount"] * np.random.uniform(5, 15), 2)
                ts = last_ts + np.random.randint(1, 120)
                last_ts = ts
                txn_time = pd.to_datetime(ts, unit="s")
            elif fraud_type == "loc_device_change":
                locs = [l for l in all_locations if l != user["home_location"]]
                location = np.random.choice(locs)
                device_id = np.random.randint(7000, 8000)
                amount = round(user["avg_amount"] * np.random.uniform(5, 15), 2)

        # Append transaction
        transactions.append({
            "transaction_id": txn_id,
            "user_id": user["user_id"],
            "transaction_amount": amount,
            "merchant_id": merchant_id,
            "device_id": device_id,
            "transaction_time": txn_time,
            "location": location,
            "is_fraud": fraud
        })
        txn_id += 1

In [23]:
upi_behavior_df = pd.DataFrame(transactions)

In [24]:
fraud_score = np.clip(
    upi_behavior_df["is_fraud"] * np.random.uniform(0.7, 1.0, len(upi_behavior_df)) +
    (1 - upi_behavior_df["is_fraud"]) * np.random.uniform(0.0, 0.3, len(upi_behavior_df)), 
    0, 1
)

In [25]:
upi_behavior_df["fraud_score"] = fraud_score

In [26]:
print(" Dataset shape:", upi_behavior_df.shape)

 Dataset shape: (75156, 9)


In [27]:
print("Users with at least 1 fraud:", upi_behavior_df.groupby("user_id")["is_fraud"].max().sum())

Users with at least 1 fraud: 715


In [28]:
print(upi_behavior_df.groupby("user_id").size().head(20))

user_id
1000    19
1001    13
1002    19
1003    11
1004    13
1005    13
1006    14
1007    17
1008    20
1009    19
1010    10
1011    10
1012    10
1013    20
1014    10
1015    20
1016    19
1017    15
1018    19
1019    12
dtype: int64


In [29]:
print(upi_behavior_df.head(15))

    transaction_id  user_id  transaction_amount  merchant_id  device_id  \
0                1     1000             1148.81          472       6706   
1                2     1000              845.45          767       6430   
2                3     1000             1164.36          570       5226   
3                4     1000              957.18          649       6138   
4                5     1000             1188.94          227       5183   
5                6     1000             1096.70          443       6394   
6                7     1000             1066.73          282       6517   
7                8     1000             1155.28          995       6719   
8                9     1000              939.56          671       5459   
9               10     1000              970.81          691       6957   
10              11     1000              785.78          853       6398   
11              12     1000            10883.86          778       5801   
12              13     10

In [30]:
upi_behavior_df.to_csv("upi.csv", index=False)